## EMR Setup with Python SDK (boto3)
This notebook will show how to set up some AWS resources using the Python SDK for AWS, boto3.

Boto3 Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html

In [ ]:
from pyspark.sql import SparkSession

aws_key      = aws_cred['default']['aws_access_key_id']
aws_secret   = aws_cred['default']['aws_secret_access_key']

spark = SparkSession \
    .builder \
    .config("spark.jars.packages","com.amazonaws:aws-java-sdk-s3:1.12.311") \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()
    

sc = spark.sparkContext
sc.setSystemProperty('com.amazonaws.services.s3.enableV4','true')

sc._jsc.hadoopConfiguration().set('fs.s3a.access.key',aws_key)
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key',aws_secret)
sc._jsc.hadoopConfiguration().set('spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled', 'true')
sc._jsc.hadoopConfiguration().set('fs.s3a.endpoint','s3-us-west-2.amazonaws.com')
sc._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled','true')

In [172]:
from pyspark.sql.types import IntegerType, StringType, FloatType, StructType, StructField

song_schema = StructType([
    StructField('num_songs',IntegerType(),True),
    StructField('artist_id',StringType(),True),
    StructField('artist_latitude',FloatType(),True),
    StructField('artist_longitude',FloatType(),True),
    StructField('artist_location',StringType(),True),
    StructField('artist_name',StringType(),True),
    StructField('song_id',StringType(),True),
    StructField('title',StringType(),True),
    StructField('duration',FloatType(),True),
    StructField('year',IntegerType(),True)
])

log_schema = StructType([
    StructField('artist',StringType(),True),
    StructField('auth',StringType(),True),
    StructField('firstName',StringType(),True),
    StructField('gender',StringType(),True),
    StructField('itemInSession',IntegerType(),True),
    StructField('lastName',StringType(),True),
    StructField('length',IntegerType(),True),
    StructField('level',StringType(),True),
    StructField('location',StringType(),True),
    StructField('method',StringType(),True),
    StructField('page',StringType(),True),
    StructField('registration',StringType(),True),
    StructField('sessionId',IntegerType(),True),
    StructField('song',StringType(),True),
    StructField('status',IntegerType(),True),
    StructField('ts',FloatType(),True),
    StructField('userAgent',StringType(),True),
    StructField('userId',StringType(),True)
])

song_df = spark.read.format('json').schema(song_schema).load('s3a://udacity-dend/song_data/*/*/*')#/A/B/C/TRABCEI128F424C983.json')
log_df = spark.read.format('json').schema(log_schema).load('s3a://udacity-dend/log_data/*/*')#/2018/11/2018-11-12-events.json')

In [171]:
log_df.toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541016707796E12,583,Sehr kosmisch,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541016707796E12,583,The Big Gundown,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541016707796E12,583,Marry Me,200,1.542243e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872073796E12,563,None,200,1.542247e+12,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541059521796E12,521,None,200,1.542253e+12,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20100101 Firefox/31.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,NaN,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266185796E12,9,Pump It,200,1.541109e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",10
8052,None,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",GET,Home,1.541016707796E12,169,None,200,1.541109e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
8053,Fall Out Boy,Logged In,Ryan,M,1,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541016707796E12,169,Nobody Puts Baby In The Corner,200,1.541109e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
8054,M.I.A.,Logged In,Ryan,M,2,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541016707796E12,169,Mango Pickle Down River (With The Wilcannia Mob),200,1.541109e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26


In [181]:
log_df = log_df.where("page = 'NextSong'")

In [173]:
from pyspark.sql.functions import udf
from datetime import datetime

get_hour = udf(lambda x: x.hour)
get_day = udf(lambda x: x.day)
get_week = udf(lambda x: x.isocalendar().week)
get_month = udf(lambda x: x.month)
get_year = udf(lambda x: x.year)
get_weekday = udf(lambda x: x.weekday())

get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000))
log_df = log_df.withColumn('timestamp',get_datetime('ts'))

In [174]:
# extract columns to create time table

from pyspark.sql import functions as F

log_df = log_df \
    .withColumn('songplay_id',F.expr("uuid()")) \
    .withColumn('hour',get_hour('timestamp')) \
    .withColumn('day',get_day('timestamp')) \
    .withColumn('week',get_week('timestamp')) \
    .withColumn('month',get_month('timestamp')) \
    .withColumn('year',get_year('timestamp')) \
    .withColumn('weekday',get_weekday('timestamp'))

In [131]:
songplays_table = songplays_table \
    .withColumn('songplay_id',F.expr("uuid()"))

In [190]:
#
#data.withColumn('timestamp',ts_to_timestamp('ts')).show()
match_condition = ((log_df.song == song_df.title) & (log_df.artist == song_df.artist_name))
songplays_table = log_df.join(song_df, match_condition, "left") \
    .select(
        log_df.ts, log_df.userId, log_df.level, log_df.sessionId,
        log_df.location, log_df.userAgent, log_df.month, log_df.year,
        song_df.song_id, song_df.artist_id
    )                


#songplays_table.limit(1).show()

In [187]:
filter_df = songplays_table.filter("song_id != 'None'").limit(5).show()

ERROR:root:KeyboardInterrupt while sending command.>               (0 + 0) / 15]
Traceback (most recent call last):
  File "/home/rambino/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/rambino/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [193]:
songplays_table \
    .select("songplay_id","ts","userId","level","song_id","artist_id","sessionId","location","userAgent","year","month") \
    .write \
    .option("header",True) \
    .partitionBy("year","month") \
    .csv("./_out/" + "songplays")   

%md

To do (Sept. 30):
1. I saved the output of a local ETL to the _out folder. Take a look at it and see if the data looks right
   1. Why are so many entries missing 'song_id' and 'artist_id'? (not all, but most)
   2. Query the data and see what kind of results I get
   3. take a look at double-checks I did for Redshift project - any I should implement here?
2. Run the etl.py again with limited data (Nov. 22 has at least 1 match in song + artist - use that?)
3. Test writing as parquet
4. Finish rest of this notebook to spin up EMR
5. Use built-in notebook to run low-data code once
6. Upload .py to EMR via SSH and run

In [150]:
%run etl.py

In [ ]:
log_df.select('artist').dropDuplicates().toPandas()

In [ ]:
from pyspark.sql.functions import desc
import pandas as pd
pd.set_option('max_colwidth', 800)

#Extract data to make songs table:
df = data.select('userId','firstName','lastName','gender','level').orderBy(desc('ts')).dropDuplicates(['userId'])
df.toPandas()
#sc._jsc.hadoopConfiguration().set('fs.s3a.endpoint','s3-us-west-2.amazonaws.com')

#writing to S3 as parquet:
# df.write \
#     .option("header",True) \
#     .partitionBy("year","artist_id") \
#     .parquet('s3a://rambino-output/test-output-parquet')

#This works ^ but it takes FOREVER. I had 23 records in this test output and it still took 26 MINUTES. Absolutely insane.
#To do:
#1. When testing my code, avoid writing to S3 until the last minute.
#2. Try to do some research as to why this is so slow and how to make it work better.

---

#### Package Import

---

In [ ]:
from asyncio import as_completed
import concurrent.futures

def func(x,y,z):
    return x+y+z

results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for cluster in [2,4]:
        futures.append(
            executor.submit(func,cluster,2,3)
        )
    for future in concurrent.futures.as_completed(futures):
        results.append(future.result())

results

In [ ]:
import boto3
import configparser

---

#### Loading Credentials from file

---

In [ ]:
#AWS Credentials
aws_path = "/home/rambino/.aws/credentials"
aws_cred = configparser.ConfigParser()
aws_cred.read(aws_path)

---

#### Create SSH keypair for connecting to EC2 instances

---

In [ ]:
ec2 = boto3.client('ec2',
    region_name             = "us-east-1",
    aws_access_key_id       = aws_cred['default']['aws_access_key_id'],
    aws_secret_access_key   = aws_cred['default']['aws_secret_access_key']
)

In [ ]:
response = ec2.create_key_pair(
    KeyName = 'spark_ec2_key',
    DryRun=False,
    KeyFormat='pem'
)

In [ ]:
with open('/home/rambino/.aws/spark_keypair.pem',"w") as file:
    file.writelines(response['KeyMaterial'])

---

#### Setting up VPC for the EMR cluster

---

If no VPC is specified for an EMR cluster, then the cluster is launched in the normal AWS cloud

Creating default VPC:

In [ ]:
!aws ec2 create-default-vpc --profile default

Getting **first** subnetId for this VPC:

In [ ]:
vpc_output = ec2.describe_vpcs()

#Getting first (and only) VPC:
vpcId = vpc_output['Vpcs'][0]['VpcId']

subnet_output = ec2.describe_subnets(
    Filters=[
        {
            'Name':'vpc-id',
            'Values':[vpcId]
        }
    ]
)

subnetId = subnet_output['Subnets'][0]['SubnetId']

---

#### Creating EMR Cluster

---

**Steps needed to set up and connect to EMR:**
1. set up cluster with correct specifications
2. get 'master public DNS' for the cluster
3. edit security group to allow my computer to connect via SSH (add inbound rule to allow SSH connection from my IP)
   1. Note: Security group is distinct entity from cluster - why not just set this up beforehand?
      1. Note: It IS possible to set up a security group beforehand - and to specify this security group for the master and slave nodes. For a more official setup, it's probably better to do this to ensure that the security group we set up for EMR is custom-defined (and not default).
      2. UPDATE: Well, actually when you CREATE a cluster, security groups are created automatically for the cluster on the default VPC. I could go through the trouble to set up custom security groups *beforehand*, or I could just create the cluster and then change the security groups as needed once they are created. Since I can't think of a reason it would be better to create custom security groups beforehand rather than just edit the ones which are created for me, I will just edit the ones created for me in this code.
4. Set up proxy to access "persistent web UI for Spark"?
   1. This looks like it's for being able to view the Spark UI somehow, but the way they're setting up the proxy settings and filtering URLs seems really hacky (e.g., they're filtering urls matching "http://10.*)". I'm not sure I want to set this up until I know that it's much better than using AWS' built-in UI viewer.
   2. Update: **It turns out that AWS also recommends using FoxyProxy (or other tools) to connect to Spark UIs on EMR**, so I will in fact do this now.
      1. [read more here](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-web-interfaces.html)

In [ ]:
emr = boto3.client('emr',
    region_name             = "us-east-1",
    aws_access_key_id       = aws_cred['default']['aws_access_key_id'],
    aws_secret_access_key   = aws_cred['default']['aws_secret_access_key']
)

In [ ]:
#With boto3
emr.run_job_flow(
            Name='spark-cluster',
            LogUri='s3://emrlogs/',
            ReleaseLabel='emr-5.28.0',
            Instances={
                'MasterInstanceType': 'm5.xlarge',
                'SlaveInstanceType': 'm5.xlarge',
                'InstanceCount': 4,
                'Ec2KeyName':'spark_ec2_key',
                'KeepJobFlowAliveWhenNoSteps': True
                #'EmrManagedMasterSecurityGroup': security_groups['manager'].id,
                #'EmrManagedSlaveSecurityGroup': security_groups['worker'].id,
            },
            Applications=[
                {
                    "Name":"Spark"
                },
                {
                    "Name":"Zeppelin"
                }
            ],
            JobFlowRole='EMR_EC2_DefaultRole',
            ServiceRole='EMR_DefaultRole',
            VisibleToAllUsers=True
        )

#NOTE: Under the 'Applications' specification of the EMR cluster above, you can also load in applications like
# Spark, TensorFlow, Presto, and Hadoop!

In [ ]:
#with AWS CLI:

!aws emr create-cluster --name test-cluster \
    --use-default-roles \
    --release-label emr-5.28.0 \
    --instance-count 4 \
    --applications Name=Spark Name=Zeppelin \
    --ec2-attributes KeyName='spark_ec2_key',SubnetId='subnet-0b6cc9cfba9463659'\
    --instance-type m5.xlarge \
    --log-uri s3://emrlogs/ \
    --visible-to-all-users

---

#### Configuring Cluster

---

In [ ]:
cluster_list = emr.list_clusters(
    ClusterStates=['STARTING','RUNNING']
)
print(cluster_list)
cluster_id = cluster_list['Clusters'][0]['Id']

In [ ]:
new_cluster = emr.describe_cluster(
    ClusterId = cluster_id
)
new_cluster
secGroup_master = new_cluster['Cluster']['Ec2InstanceAttributes']['EmrManagedMasterSecurityGroup']
cluster_dns = new_cluster['Cluster']['MasterPublicDnsName']


Configure Cluster Security Groups to only accept SSH ingress from my IP address

In [ ]:
#Getting my public IP address from config.me website (IP is last element of returned array)
myIP = !curl ifconfig.me
myIP = myIP[-1]

In [ ]:
#Specifying internal port (arbitrary?)
myPort = '32'
myCidrIp = myIP + "/" + myPort

In [ ]:
response = ec2.authorize_security_group_ingress(
    GroupId=secGroup_master,
    IpPermissions=[
        {
            'FromPort': 22,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': myCidrIp,
                    'Description': 'SSH access to Spark EMR on AWS from Kevins Computer',
                },
            ],
            'ToPort': 22,
        },
    ],
)

---

#### Interacting with Cluster

---

In [ ]:
#File path where cluster login information is kept on my machine:
pem_path = '/home/rambino/.aws/spark_keypair.pem'

Connect to Cluster via SSH

In [ ]:
#Command to use in terminal (interactive):
print(f"ssh hadoop@{cluster_dns} -i {pem_path}")

---

#### Proxy connection to allow interaction with Spark UI

---

Setting up FoxyProxy to allow connection to Spark UI from localhost
[AWS Documentation on Port forwarding for EMR connections](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-ssh-tunnel.html)


I needed to install the browser extension FoxyProxy to allow my browser to interface with the EMR cluster. Once I installed it, I then needed to set up a new proxy with these settings:
- IP address: `localhost`
- Port: `8157` (only needed to match dynamic port forwarding below)

Then, in the 'pattern matching' part, I needed to specify which URLs should be forwarded in this way. This was already specified by Udacity. The json file accompanying this notebook named 'foxyproxy...' shows these patterns.

In [ ]:
#Copying credentials file to the master node (not sure why yet)
print(f"scp -i {pem_path} {pem_path} hadoop@{cluster_dns}:/home/hadoop/")

In [ ]:
#This sets up port forwarding (somehow) so that data from our local machine on port 8157 is forwarded to the master node (allowing interactivity)
#NOTE: Terminal remains open when this request succeeds - and needs to remain running while accessing Spark UI

#Note: Getting this SSH connection to work has been unpredictable at times. Often get 'connection refused' errors, but then it
#suddenly works. Should ideally figure out what's going on there...

print(f"ssh -v -i {pem_path} -N -D 127.0.0.1:8157 hadoop@{cluster_dns}")

#### Accessing Spark UI:
- Base URL:           http://ec2-54-87-42-167.compute-1.amazonaws.com

- Spark History:      http://ec2-54-87-42-167.compute-1.amazonaws.com:18080/
- YARN Node Manager:  http://ec2-54-87-42-167.compute-1.amazonaws.com:8042/


[See more ports here](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-web-interfaces.html)

---

#### Deleting EMR Cluster (Teardown)

---

In [ ]:
emr.terminate_job_flows(
    JobFlowIds=[
        cluster_id
    ]
)